In [97]:
from mxnet import gluon
from mxnet import ndarray as nd

def transform(data, label):
    return data.astype('float32')/255, label.astype('float32')
mnist_train = gluon.data.vision.FashionMNIST(train = True, transform = transform)
mnist_test = gluon.data.vision.FashionMNIST(train = False, transform = transform)

In [98]:
batch_size = 256
train_data = gluon.data.DataLoader(mnist_train, batch_size, shuffle= True)
test_data = gluon.data.DataLoader(mnist_test, batch_size, shuffle=False)

In [99]:
num_inputs = 28 * 28
num_outputs = 10
num_hidden = 256
weight_scale = 0.01
#在这里为什么对0.1的标准差进行初始化，其训练结果有什么区别
#对b初始化为0或者采样进行比较
W1 = nd.random.normal(shape = (num_inputs, num_hidden), scale = weight_scale)
# b1 = nd.random.normal(shape = num_hidden, scale = weight_scale)
b1 = nd.zeros(num_hidden)
W2 = nd.random.normal(shape = (num_hidden, num_outputs), scale = weight_scale)
# b2 = nd.random.normal(shape = num_outputs, scale = weight_scale)
b2 = nd.zeros(num_outputs)
params = [W1, b1, W2, b2]
for param in params:
    param.attach_grad()

### 激活函数
可以进行非线性函数拟合，这里使用RELU
                relu(x) = max(x,0)

In [100]:
def relu(x):
    return nd.maximum(x,0)

In [101]:
def net(x):
    x = x.reshape((-1, num_inputs))
    h1 = relu(nd.dot(x, W1) + b1)
    output = nd.dot(h1, W2) + b2
    return output

In [102]:
def SGD(params, lr):
    for param in params:
        param[:] = param - lr * param.grad 

### softmax和交叉熵
使用gluon里面带的softmax和交叉熵

In [103]:
from mxnet import gluon
softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()

In [106]:
from mxnet import autograd
learning_rate = 0.5
for e in range(5):
    train_loss = 0
    for data,label in train_data:
        with autograd.record():
            output = net(data)
            loss = softmax_cross_entropy(output, label)
        loss.backward()
        SGD(params, learning_rate/batch_size)
        
        train_loss += nd.mean(loss).asscalar()
    print("epoch: %d, train loss: %f" % (e, train_loss/len(train_data)))

epoch: 0, train loss: 0.297390
epoch: 1, train loss: 0.288182
epoch: 2, train loss: 0.281299
epoch: 3, train loss: 0.275299
epoch: 4, train loss: 0.266363


### 练习
尝试改变 weight_scale会有什么变化
尝试改变num_hidden会有什么变化
如果多加入一个隐含层会有什么变化